In [ ]:
import cv2
import numpy as np

def estimate_veiling(image):
    dark_prior = np.min(image, axis=2)
    veiling = 1 - dark_prior / 255  # 
    cv2.imshow('Estimated Veiling', (veiling * 255).astype(np.uint8))
    cv2.waitKey(0)
    return veiling
def adaptive_wiener_filter(observation, local_mean, local_variance, noise_variance):
    refined_estimate = local_mean + ((local_variance - noise_variance) / local_variance) * (observation - local_mean)
    return refined_estimate
def calculate_local_variance(image, window_size):
    height, width = image.shape
    pad = window_size // 2
    padded_image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REFLECT)
    local_variance = np.zeros_like(image, dtype=np.float32)
    for y in range(pad, height + pad):
        for x in range(pad, width + pad):
            window = padded_image[y - pad:y + pad + 1, x - pad:x + pad + 1]
            local_mean = np.mean(window)
            local_variance[y - pad, x - pad] = np.mean((window - local_mean) ** 2)
    return local_variance
def estimate_noise_variance(image):
    observation_variances = np.var(image, axis=(0, 1))
    noise_variance = np.mean(observation_variances) / (image.shape[0] * image.shape[1])    
    return noise_variance
def defog_image(image, veiling, noise_variance):
    window_size = 32
    local_mean = cv2.blur(veiling, (window_size, window_size))
    local_variance = calculate_local_variance(veiling, window_size)
    refined_veiling = adaptive_wiener_filter(veiling, local_mean, local_variance, noise_variance)
    cv2.imshow('Refined Veiling Estimate', (refined_veiling * 255).astype(np.uint8))
    cv2.waitKey(0)
    transmission = 1 - refined_veiling
    cv2.imshow('Transmission Map', (transmission * 255).astype(np.uint8))
    cv2.waitKey(0)
    defogged_image = (image - transmission[..., None] * 255) / np.maximum(transmission[..., None], 0.01) + transmission[..., None] * 255
    defogged_image = np.clip(defogged_image, 0, 255).astype(np.uint8)
    cv2.imshow('Defogged Image', defogged_image)
    cv2.waitKey(0)
    return defogged_image

foggy_image = cv2.imread('F1.png')
cv2.imshow('Foggy Image', foggy_image)
cv2.waitKey(0)
# Estimate veiling from the foggy image
veiling_estimate = estimate_veiling(foggy_image)
# Decompose the foggy image into low and high-resolution components
low_res_image = cv2.pyrDown(foggy_image)
high_res_image = foggy_image - cv2.pyrUp(low_res_image)
# Estimate noise variance using the low-resolution component
noise_variance = estimate_noise_variance(low_res_image)
# Defog the image with the estimated veiling and noise variance
defogged_image = defog_image(foggy_image, veiling_estimate, noise_variance)

cv2.destroyAllWindows()